In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\ksalm\\OneDrive\\Desktop\\krish_resume_projects\\NLP\\Text-Summarizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_dir: Path
    unzip_data_dir: Path

In [5]:
from src.textSummarizer.constant import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH, 
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        return DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_URL = config.source_URL,
            local_data_dir = Path(config.local_data_dir),
            unzip_data_dir = Path(config.unzip_data_dir)
        )

In [7]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self):
        ''' 
        downloads the data from the source URL
        '''
        if not os.path.exists(self.config.local_data_dir):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, 
                filename = self.config.local_data_dir)
            logger.info(f"{filename} download! with following information: \n{headers}")
        else:
            logger.info(f"File already exists of size {get_size(Path(self.config.local_data_dir))}")
            
    def unzip_data(self):
        '''
        extracts the zip file to the specified directory
        '''
        unzip_data_dir = self.config.unzip_data_dir
        # create directory if not exists
        os.makedirs(unzip_data_dir, exist_ok=True)
        
        print(self.config.local_data_dir)
        # unzip data
        with zipfile.ZipFile(self.config.local_data_dir, 'r') as zip_ref:
            zip_ref.extractall(unzip_data_dir)
            logger.info(f"Unzipped data at {unzip_data_dir}")

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_data()
except Exception as e:
    logger.error(f"Error in data ingestion: {str(e)}")

src\textSummarizer\config\config.yaml
[2025-01-05 19:25:38,736 : INFO : common : yaml file: src\textSummarizer\config\config.yaml loaded successfully]
params\params.yaml
[2025-01-05 19:25:38,740 : INFO : common : yaml file: params\params.yaml loaded successfully]
[2025-01-05 19:25:38,742 : INFO : common : Created directory at: artifacts]
[2025-01-05 19:25:38,744 : INFO : common : Created directory at: artifacts/data_ingestion]
[2025-01-05 19:25:40,721 : INFO : 2367143137 : artifacts\data_ingestion\samsumdata.zip download! with following information: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D96B:291D25:2AF9F7:3053F2:677B3113
Accep